# Análisis Exploratorio de Datos (EDA) - Producción Agroindustrial de Ica

**Objetivo**: Realizar un análisis exploratorio completo del dataset de certificaciones de productos agrícolas de la región de Ica (Perú).

**Fuente de datos**: Certificaciones VUCE (Ventanilla Única de Comercio Exterior) - SENASA

**Período**: 2025 (datos más recientes)

---

## 1. Importación de Librerías y Configuración

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from pathlib import Path

# Configuración
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Configurar directorio de trabajo
import os
os.chdir('..')

print("📊 Librerías importadas correctamente")
print("📁 Directorio de trabajo:", os.getcwd())

## 2. Carga de Datos

In [ ]:
# Cargar datos
df = pd.read_csv("data/lugar_produccion_ica.csv", encoding='latin-1')

print(f"✅ Dataset cargado exitosamente")
print(f"📊 Dimensiones: {df.shape[0]} filas, {df.shape[1]} columnas")
print("\n📋 Primeras 5 filas:")
df.head()

## 3. Exploración Inicial

In [ ]:
# Información general del dataset
print("📋 INFORMACIÓN GENERAL DEL DATASET")
print("=" * 50)

print(f"\n📊 Información básica:")
print(f"- Período: {df['AÑO CERTIFICACION'].min()} - {df['AÑO CERTIFICACION'].max()}")
print(f"- Departamento: {df['DEPARTAMENTO'].unique()}")
print(f"- Provincias: {len(df['PROVINCIA'].unique())} únicas")
print(f"- Distritos: {len(df['DISTRITO'].unique())} únicos")
print(f"- Productos únicos: {len(df['CULTIVO'].unique())}")
print(f"- Tipos de productor: {df['TIPO PROD'].unique()}")

# Información de columnas
print("\n📋 Columnas del dataset:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

In [ ]:
# Análisis de tipos de datos y valores nulos
print("📊 ANÁLISIS DE CALIDAD DE DATOS")
print("=" * 50)

# Crear resumen de calidad
calidad_datos = pd.DataFrame({
    'Tipo_Datos': df.dtypes,
    'Valores_Nulos': df.isnull().sum(),
    'Porcentaje_Nulos': (df.isnull().sum() / len(df) * 100).round(2),
    'Valores_Únicos': df.nunique()
})

print("\n📋 Resumen de calidad de datos:")
display(calidad_datos)

## 4. Análisis Descriptivo

In [ ]:
# Estadísticas descriptivas de variables numéricas
print("📊 ESTADÍSTICAS DESCRIPTIVAS")
print("=" * 50)

# Columnas numéricas principales
numeric_cols = ['AREA TOTAL (HA)', 'AREA CERTIFICAR (HA)', 'RENDIMIENTO CERTIFICADO', 'RENDIMIENTO SOLICITUD']
existing_numeric_cols = [col for col in numeric_cols if col in df.columns]

if existing_numeric_cols:
    # Convertir a numérico
    for col in existing_numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    print("\n📋 Estadísticas descriptivas (variables numéricas):")
    display(df[existing_numeric_cols].describe())
else:
    print("⚠️ No se encontraron columnas numéricas específicas")

In [ ]:
# Top 10 productos más frecuentes
print("🥇 TOP 10 PRODUCTOS MÁS CERTIFICADOS")
print("=" * 50)

top_productos = df['CULTIVO'].value_counts().head(10)
print("\n📋 Ranking de productos:")
for i, (producto, count) in enumerate(top_productos.items(), 1):
    print(f"{i:2d}. {producto}: {count:,} certificaciones")

# Visualización
plt.figure(figsize=(12, 8))
sns.barplot(data=top_productos.reset_index(), y='CULTIVO', x='count', palette='viridis')
plt.title('Top 10 Productos Más Certificados en Ica', fontsize=16, fontweight='bold')
plt.xlabel('Número de Certificaciones', fontsize=12)
plt.ylabel('Producto', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Distribución geográfica por provincia y distrito
print("🗺️ DISTRIBUCIÓN GEOGRÁFICA")
print("=" * 50)

# Por provincia
provincia_dist = df['PROVINCIA'].value_counts()
print("\n📋 Distribución por provincia:")
for provincia, count in provincia_dist.items():
    print(f"- {provincia}: {count:,} certificaciones ({count/len(df)*100:.1f}%)")

# Top 10 distritos
print("\n📋 Top 10 distritos más activos:")
top_distritos = df['DISTRITO'].value_counts().head(10)
for i, (distrito, count) in enumerate(top_distritos.items(), 1):
    print(f"{i:2d}. {distrito}: {count:,} certificaciones")

# Visualización por provincia
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico de barras por provincia
provincia_dist.plot(kind='bar', ax=ax1, color='skyblue')
ax1.set_title('Distribución de Certificaciones por Provincia', fontsize=14, fontweight='bold')
ax1.set_ylabel('Número de Certificaciones')
ax1.tick_params(axis='x', rotation=45)

# Gráfico de pastel por provincia
provincia_dist.plot(kind='pie', ax=ax2, autopct='%1.1f%%', startangle=90)
ax2.set_title('Distribución Porcentual por Provincia', fontsize=14, fontweight='bold')
ax2.set_ylabel('')

plt.tight_layout()
plt.show()

## 5. Análisis Temporal

In [ ]:
# Análisis temporal por mes
print("📅 ANÁLISIS TEMPORAL")
print("=" * 50)

# Distribución mensual
mes_dist = df['MES CERTIFICACION'].value_counts().sort_index()
print("\n📋 Distribución mensual de certificaciones:")
for mes, count in mes_dist.items():
    print(f"- {mes}: {count:,} certificaciones")

# Visualización temporal
plt.figure(figsize=(14, 8))

# Tendencia mensual
plt.subplot(2, 2, 1)
mes_dist.plot(kind='bar', color='coral')
plt.title('Certificaciones por Mes', fontsize=12, fontweight='bold')
plt.ylabel('Número de Certificaciones')
plt.tick_params(axis='x', rotation=45)

# Análisis por tipo de productor
plt.subplot(2, 2, 2)
tipo_prod_dist = df['TIPO PROD'].value_counts()
tipo_prod_dist.plot(kind='pie', autopct='%1.1f%%', startangle=90)
plt.title('Distribución por Tipo de Productor', fontsize=12, fontweight='bold')
plt.ylabel('')

# Heatmap mensual por provincia
plt.subplot(2, 1, 2)
heatmap_data = df.groupby(['PROVINCIA', 'MES CERTIFICACION']).size().unstack(fill_value=0)
sns.heatmap(heatmap_data, annot=True, fmt='d', cmap='YlOrRd', cbar_kws={'label': 'Certificaciones'})
plt.title('Mapa de Calor: Certificaciones por Provincia y Mes', fontsize=12, fontweight='bold')
plt.xlabel('Mes')
plt.ylabel('Provincia')

plt.tight_layout()
plt.show()

## 6. Análisis de Rendimiento y Área

In [ ]:
# Análisis de rendimiento por cultivo
print("🌱 ANÁLISIS DE RENDIMIENTO Y ÁREA")
print("=" * 50)

# Estadísticas por cultivo (para cultivos con más de 5 registros)
cultivo_stats = df.groupby('CULTIVO').agg({
    'AREA CERTIFICAR (HA)': ['count', 'sum', 'mean'],
    'RENDIMIENTO CERTIFICADO': ['mean', 'std']
}).round(2)

# Filtrar cultivos con más de 5 registros
cultivo_stats_filtered = cultivo_stats[cultivo_stats[('AREA CERTIFICAR (HA)', 'count')] >= 5]

print("\n📋 Estadísticas por cultivo (>5 registros):")
print("Cultivo | Certificaciones | Área Total (Ha) | Área Promedio | Rendimiento Promedio")
print("-" * 90)

for cultivo in cultivo_stats_filtered.index:
    count = cultivo_stats_filtered.loc[cultivo, ('AREA CERTIFICAR (HA)', 'count')]
    area_total = cultivo_stats_filtered.loc[cultivo, ('AREA CERTIFICAR (HA)', 'sum')]
    area_prom = cultivo_stats_filtered.loc[cultivo, ('AREA CERTIFICAR (HA)', 'mean')]
    rend_prom = cultivo_stats_filtered.loc[cultivo, ('RENDIMIENTO CERTIFICADO', 'mean')]
    
    print(f"{cultivo[:25]:<25} | {count:>12} | {area_total:>12.1f} | {area_prom:>11.1f} | {rend_prom:>17.1f}")

In [ ]:
# Visualización de rendimiento vs área
# Preparar datos para visualización
cultivo_viz = df.groupby('CULTIVO').agg({
    'AREA CERTIFICAR (HA)': ['count', 'sum'],
    'RENDIMIENTO CERTIFICADO': 'mean'
}).round(2)

cultivo_viz.columns = ['Certificaciones', 'Area_Total', 'Rendimiento_Promedio']
cultivo_viz = cultivo_viz[cultivo_viz['Certificaciones'] >= 3].reset_index()

# Crear visualización interactiva con Plotly
fig = px.scatter(
    cultivo_viz, 
    x='Certificaciones', 
    y='Rendimiento_Promedio',
    size='Area_Total',
    hover_name='CULTIVO',
    hover_data=['Area_Total'],
    title='Rendimiento vs Frecuencia de Certificaciones por Cultivo',
    labels={
        'Certificaciones': 'Número de Certificaciones',
        'Rendimiento_Promedio': 'Rendimiento Promedio (t/ha)',
        'Area_Total': 'Área Total (Ha)'
    },
    color='Rendimiento_Promedio',
    color_continuous_scale='viridis'
)

fig.update_layout(
    width=900,
    height=600,
    title_font_size=16
)

fig.show()

## 7. Análisis por Tipo de Productor

In [ ]:
# Análisis por tipo de productor
print("👥 ANÁLISIS POR TIPO DE PRODUCTOR")
print("=" * 50)

# Distribución general
tipo_prod_dist = df['TIPO PROD'].value_counts()
print("\n📋 Distribución por tipo de productor:")
for tipo, count in tipo_prod_dist.items():
    print(f"- {tipo}: {count:,} certificaciones ({count/len(df)*100:.1f}%)")

# Análisis por cultivo y tipo de productor
tipo_cultivo_cross = pd.crosstab(df['CULTIVO'], df['TIPO PROD'])
top_cultivos = df['CULTIVO'].value_counts().head(8).index
tipo_cultivo_top = tipo_cultivo_cross.loc[top_cultivos]

print("\n📋 Distribución por tipo de productor en principales cultivos:")
display(tipo_cultivo_top)

# Visualización
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico de barras apiladas
tipo_cultivo_top.plot(kind='bar', stacked=True, ax=ax1, color=['lightblue', 'lightcoral'])
ax1.set_title('Tipo de Productor por Cultivo Principal', fontsize=14, fontweight='bold')
ax1.set_ylabel('Número de Certificaciones')
ax1.tick_params(axis='x', rotation=45)
ax1.legend(title='Tipo Productor')

# Gráfico de barras agrupadas
tipo_cultivo_top.plot(kind='bar', ax=ax2, color=['lightblue', 'lightcoral'])
ax2.set_title('Comparación por Tipo de Productor', fontsize=14, fontweight='bold')
ax2.set_ylabel('Número de Certificaciones')
ax2.tick_params(axis='x', rotation=45)
ax2.legend(title='Tipo Productor')

plt.tight_layout()
plt.show()

## 8. Insights y Conclusiones Principales

In [ ]:
# Generar insights principales
print("🎯 INSIGHTS Y CONCLUSIONES PRINCIPALES")
print("=" * 60)

# Calcular métricas clave
total_certificaciones = len(df)
total_area = df['AREA CERTIFICAR (HA)'].sum()
productos_unicos = df['CULTIVO'].nunique()
distritos_unicos = df['DISTRITO'].nunique()
provincias_unicas = df['PROVINCIA'].nunique()

producto_principal = df['CULTIVO'].value_counts().index[0]
provincia_principal = df['PROVINCIA'].value_counts().index[0]
distrito_principal = df['DISTRITO'].value_counts().index[0]
mes_principal = df['MES CERTIFICACION'].value_counts().index[0]

print(f"\n📊 MÉTRICAS CLAVE:")
print(f"- Total de certificaciones: {total_certificaciones:,}")
print(f"- Área total certificada: {total_area:,.1f} hectáreas")
print(f"- Productos únicos: {productos_unicos}")
print(f"- Distritos participantes: {distritos_unicos}")
print(f"- Provincias participantes: {provincias_unicas}")

print(f"\n🥇 PRINCIPALES HALLAZGOS:")
print(f"- Producto más certificado: {producto_principal}")
print(f"- Provincia más activa: {provincia_principal}")
print(f"- Distrito más activo: {distrito_principal}")
print(f"- Mes con más certificaciones: {mes_principal}")

# Tipo de productor predominante
tipo_predominante = df['TIPO PROD'].value_counts().index[0]
porcentaje_tipo = (df['TIPO PROD'].value_counts().iloc[0] / len(df)) * 100
print(f"- Tipo de productor predominante: {tipo_predominante} ({porcentaje_tipo:.1f}%)")

print(f"\n🔍 INSIGHTS ESTRATÉGICOS:")
print(f"1. La región de Ica se enfoca principalmente en cultivos de exportación")
print(f"2. {provincia_principal} es la provincia líder en certificaciones")
print(f"3. Los productores jurídicos (empresas) dominan el sector")
print(f"4. La estacionalidad muestra picos en {mes_principal}")
print(f"5. {producto_principal} es el cultivo estrella de la región")
print(f"6. La diversificación productiva incluye {productos_unicos} cultivos diferentes")
print(f"7. La cobertura geográfica abarca {distritos_unicos} distritos")

print(f"\n💡 RECOMENDACIONES:")
print(f"- Fortalecer la cadena productiva de {producto_principal}")
print(f"- Apoyar el desarrollo de pequeños productores (tipo Natural)")
print(f"- Expandir la certificación en distritos con menor participación")
print(f"- Desarrollar estrategias de diversificación de cultivos")
print(f"- Optimizar los procesos de certificación en época de alta demanda")

## 9. Exportación de Resultados

In [ ]:
# Crear resumenes para exportar
print("💾 EXPORTANDO RESULTADOS")
print("=" * 50)

# Crear directorio de salida
output_dir = Path('outputs')
output_dir.mkdir(exist_ok=True)

# Resumen ejecutivo
resumen_ejecutivo = {
    'Métrica': ['Total Certificaciones', 'Área Total (Ha)', 'Productos Únicos', 'Distritos', 'Provincias'],
    'Valor': [total_certificaciones, f"{total_area:,.1f}", productos_unicos, distritos_unicos, provincias_unicas]
}

df_resumen = pd.DataFrame(resumen_ejecutivo)
df_resumen.to_csv(output_dir / 'resumen_ejecutivo.csv', index=False, encoding='utf-8')

# Top productos
top_productos.to_csv(output_dir / 'top_productos.csv', header=['Certificaciones'], encoding='utf-8')

# Distribución geográfica
distribucion_geo = pd.DataFrame({
    'Provincia': provincia_dist.index,
    'Certificaciones': provincia_dist.values,
    'Porcentaje': (provincia_dist.values / len(df) * 100).round(1)
})
distribucion_geo.to_csv(output_dir / 'distribucion_geografica.csv', index=False, encoding='utf-8')

print("✅ Resultados exportados exitosamente")
print(f"📁 Archivos guardados en: {output_dir.absolute()}")
print("- resumen_ejecutivo.csv")
print("- top_productos.csv")
print("- distribucion_geografica.csv")

---

## Conclusiones del Análisis Exploratorio

Este análisis exploratorio ha revelado aspectos importantes sobre la producción agroindustrial certificada en la región de Ica:

### 🎯 Puntos Clave:
1. **Diversidad productiva**: La región cuenta con múltiples cultivos certificados
2. **Concentración geográfica**: Ciertas provincias y distritos lideran la actividad
3. **Estacionalidad**: Patrones claros de certificación por meses
4. **Estructura empresarial**: Predominio de productores jurídicos
5. **Potencial de crecimiento**: Oportunidades en distritos menos participativos

### 📈 Próximos Pasos:
- Análisis de modelado predictivo (Notebook 02)
- Desarrollo de indicadores de competitividad
- Análisis de correlaciones entre variables
- Segmentación de productores

---
*Análisis realizado con Python - Pandas, Seaborn, Plotly*